In [1]:
import numpy as np
import pandas as pd
import keras
import os
from pathlib import Path

In [2]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [3]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 5000  # Number of samples to train on.

In [4]:
data_path = "fra.txt"

In [5]:
input_texts = []
target_texts = []
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = target_text
    input_texts.append(input_text)
    target_texts.append(target_text)

In [6]:
exclude = set(string.punctuation)

In [7]:
remove_digits = str.maketrans('', '', digits)

In [8]:
input_data = []
for i in range(0, len(input_texts)):
    review = input_texts[i]
    review = review.lower()
    review = re.sub("'", '', review)
    review = re.sub(",", ' COMMA', review) 
    review = [ch for ch in review if ch not in exclude]
    review = ''.join(review)
    review = review.translate(remove_digits)
    input_data.append(review)

In [9]:
target_data = []
for i in range(0, len(target_texts)):
    review = target_texts[i]
    review = review.lower()
    review = re.sub("'", '', review)
    review = re.sub(",", ' COMMA', review)
    review = [ch for ch in review if ch not in exclude]
    review = ''.join(review)
    review = review.translate(remove_digits)
    review = 'START_ '+ review + ' _END'
    target_data.append(review)

In [10]:
target_data[0]

'START_ va  _END'

In [11]:
english_words=set()
for eng in input_data:
    for word in eng.split():
        if word not in english_words:
            english_words.add(word)

In [12]:
french_words=set()
for french in target_data:
    for word in french.split():
        if word not in french_words:
            french_words.add(word)

In [13]:
print(len(english_words))
print(len(french_words))

1107
2583


In [14]:
lenght_list_english=[]
for l in input_data:
    lenght_list_english.append(len(l.split(' ')))
np.max(lenght_list_english)

4

In [15]:
lenght_list_french=[]
for l in target_data:
    lenght_list_french.append(len(l.split(' ')))
np.max(lenght_list_french)

12

In [16]:
all_english_words = sorted(list(english_words))
all_french_words = sorted(list(french_words))
english_vocab_size = len(all_english_words)
french_vocab_size = len(all_french_words)
english_max_sent_length = np.max(lenght_list_english)
french_max_sent_length = np.max(lenght_list_french)

In [17]:
english_token_index = dict([(word, i) for i, word in enumerate(all_english_words)])
french_token_index = dict([(word, i) for i, word in enumerate(all_french_words)])

In [18]:
encoder_input_data = np.zeros((num_samples, english_max_sent_length, english_vocab_size), dtype='float32')
decoder_input_data = np.zeros((num_samples, french_max_sent_length, french_vocab_size), dtype='float32')
decoder_target_data = np.zeros((num_samples, french_max_sent_length, french_vocab_size), dtype='float32')

In [19]:
for i, (input_text, target_text) in enumerate(zip(input_data, target_data)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t, english_token_index[word]] = 1.
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, french_token_index[word]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, french_token_index[word]] = 1.

In [20]:
#building the model
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, english_vocab_size))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [21]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, french_vocab_size))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(french_vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [22]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [23]:
#training the model
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.05,
)
# Save model
model.save("s2s_model.keras")

Epoch 1/10
75/75 [==============================] - 48s 569ms/step - loss: 1.7068 - accuracy: 0.0822 - val_loss: 1.8838 - val_accuracy: 0.0833
Epoch 2/10
75/75 [==============================] - 35s 471ms/step - loss: 1.5964 - accuracy: 0.0956 - val_loss: 1.9374 - val_accuracy: 0.0837
Epoch 3/10
75/75 [==============================] - 35s 463ms/step - loss: 1.5753 - accuracy: 0.0966 - val_loss: 1.9937 - val_accuracy: 0.0837
Epoch 4/10
75/75 [==============================] - 34s 454ms/step - loss: 1.5486 - accuracy: 0.0967 - val_loss: 2.0177 - val_accuracy: 0.0837
Epoch 5/10
75/75 [==============================] - 34s 454ms/step - loss: 1.5136 - accuracy: 0.1005 - val_loss: 2.0113 - val_accuracy: 0.0870
Epoch 6/10
75/75 [==============================] - 34s 459ms/step - loss: 1.4784 - accuracy: 0.1075 - val_loss: 2.0298 - val_accuracy: 0.0877
Epoch 7/10
75/75 [==============================] - 34s 456ms/step - loss: 1.4493 - accuracy: 0.1110 - val_loss: 2.0229 - val_accuracy: 0.0897

In [24]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in english_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in french_token_index.items())

In [25]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, french_vocab_size))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, french_token_index['START_']] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == '_END' or len(decoded_sentence) > french_max_sent_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, french_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [26]:
for seq_index in [1,20,30,56,199]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_data[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence[:-4])

-
Input sentence: ['go']
Decoded sentence:  prends 
-
Input sentence: ['run']
Decoded sentence:  laissezmoi 
-
Input sentence: ['help']
Decoded sentence:  laissezmoi 
-
Input sentence: ['i see']
Decoded sentence:  je 
-
Input sentence: ['beat it']
Decoded sentence:  laissezmoi 
